In [ ]:
import requests
import csv
import os
import time
from dotenv import load_dotenv
from groq import Groq

load_dotenv()

# API Key của Groq
GROQ_API_KEY = os.getenv("Điền vào đây")
client = Groq(api_key="Điền vào đây")

def generate_answer_groq(prompt):
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-70b-8192",  # Chỉnh sửa nếu cần
        temperature=0,
        max_tokens=1024,
    )
    return chat_completion.choices[0].message.content

def generate_labels(input_file, output_file, pause_seconds=20):
    try:
        with open(input_file, 'r', encoding='utf-8') as infile, \
             open(output_file, 'w', encoding='utf-8', newline='') as outfile:
            
            reader = csv.reader(infile, delimiter='\t')
            writer = csv.writer(outfile, delimiter='\t')
            
            # Giả sử file đầu vào có tiêu đề, thêm tiêu đề cho file đầu ra
            headers = next(reader, None)
            if headers:
                writer.writerow(headers + ['Labels'])
            else:
                writer.writerow(['Content', 'Labels'])
            
            for idx, row in enumerate(reader, 1):
                if not row or len(row) < 1:
                    writer.writerow(["Không xác định"])
                    continue  # Bỏ qua dòng trống
                
                text = row[0].strip()  # Giả sử cột đầu tiên là nội dung văn bản
                if not text:
                    writer.writerow([row[0], "Không xác định"])
                    continue
                
                # Thiết kế prompt cho mô hình Llama
                prompt = f"""Bạn là một mô hình trí tuệ nhân tạo chuyên về phân loại văn bản. Với đoạn văn bản dưới đây, 
                hãy tự động tạo ra các nhãn (keywords) phản ánh nội dung chính của đoạn văn bản. Các nhãn phải được viết bằng tiếng Việt,
                ngắn gọn (tối đa 3 từ), chính xác và được tách nhau bằng dấu phẩy. Tránh các nhãn quá chung chung hoặc không liên quan,
                ngoài việc trả lời các nhãn ra thì không giải thích gì hơn, chỉ cần trả ra nhãn và viết trên cùng 1 dòng không được thêm bất cứ điều gì.
    ### Ví dụ 1:
    **Văn bản:** "Bước vào trận đấu, Barcelona nhanh chóng tràn lên tấn công nhưng vấp phải sự kháng cự quyết liệt của Sevilla. 
    Bước ngoặt đến ở phút 22, Raphinha bị phạm lỗi trong vòng cấm và trọng tài đã cho Barcelona hưởng quả phạt đền. Trên chấm 11m,
    Lewandowski đã ghi bàn mở tỷ số cho Barcelona. Đến phút 28, Pedri đã có bàn nhân đôi cách biệt cho đội chủ nhà trước khi 
    Lewandowski hoàn tất cú đúp cho riêng mình ở phút 39. Hiệp 1 khép lại với tỉ số 3-0 nghiêng về Barcelona. Sang hiệp 2,
    các chân sút của Barcelona liên tiếp bắn phá khung thành của Sevilla nhưng rất tiếc đều rơi vào thế việt vị. Tuy nhiên,
    chỉ trong vòng ít phút từ phút 82 đến 88, các khán giả có mặt trên sân Olimpic Lluis Companys đã được chứng kiến tới 
    3 bàn thắng. Cầu thủ vào sân thay người bên phía Barcelona là Torre lập cú đúp, xen giữa là bàn thắng danh dự của Idumbo 
    bên phía Sevilla. Chung cuộc, Barca thắng Sevilla với tỷ số 5-1. Như vậy, đoàn quân của HLV Hansi Flick củng cố vị trí 
    dẫn đầu trên bảng xếp hạng La Liga, duy trì khoảng cách 3 điểm với đại kình địch Real Madrid."
    **Nhãn:** La Liga,Sevilla,Olimpic Lluis Companys,đè bẹp,Lewandowski,Barcelona,Raphinha,Pedri,Hansi Flick,tiếp đón,primera,
    củng cố vị trí,kháng cự,bắn phá,vào sân thay người,nhân đôi cách biệt,tỏa sáng,xen,sân nhà,chấm 11m
    
    ### Ví dụ 2:
    **Văn bản:** "Willian đi vào lịch sử bóng đá xứ samba. Với 1 bàn thắng và 1 pha kiến tạo trong chiến thắng 5-3 của Palmeiras
    trước Juventude ở vòng 30 Serie A 2024/25 hôm 21/10, Willian cán mốc 10 bàn và 8 kiến tạo ngay trong mùa giải đầu tiên chơi
    ở hạng đấu cao nhất Brazil. Tài năng trẻ này trở thành cầu thủ U17 in dấu giày trực tiếp vào bàn thắng nhiều nhất trong một
    mùa giải của Serie A. Kỷ lục trước đó của giải đấu được giữ bởi Neymar, khi cựu sao Santos ghi 10 bàn và có 6 kiến tạo trong
    cả mùa giải 2008/09. Ở thời điểm đó, tài năng trẻ 17 tuổi mang tên Neymar khiến cả Brazil phát cuồng. Tuy nhiên, Willian hiện
    có thông số tốt hơn Neymar khi cả hai cùng ở tuổi 17. Estevao ra mắt Serie A Brazil hồi đầu năm, nhưng nhanh chóng trở thành 
    hiện tượng, khi chiếm suất đá chính tại đội đương kim vô địch Palmeiras. “Kể từ sau Neymar, Estevao Wilian là cầu thủ sinh 
    ra ở Brazil hay nhất mà tôi từng thấy”, Branco, thành viên tuyển Brazil vô địch World Cup 1994 bình luận trên Wamo Cast. 
    Ông tiếp tục: “Với tất cả sự tôn trọng dành cho những người khác, mọi thứ tôi nói hoàn toàn nghiêm túc". Ở tuổi 17, 
    Willian khuấy đảo các hàng thủ tại giải VĐQG Brazil, điều hiếm thấy trong lịch sử. Chính Neymar cũng phải thừa nhận 
    "Willian là tài năng đang nổi và tôi nghĩ cậu ấy sẽ trở thành một thiên tài”. Chelsea đang rất vui với phong độ hiện 
    tại của Estevao Willian. "The Blues" hoàn tất thương vụ mua cầu thủ với giá 65 triệu euro hồi tháng trước. Tiền đạo 
    người Brazil sẽ khoác áo Chelsea vào năm 2025, khi đủ 18 tuổi."
    **Nhãn:** Estevao Willian,Neymar,giải VĐQG Brazil,Serie A 2024/25,Palmeiras,U17,Santos,kiến tạo,tuyển Brazil,World Cup 1994,Serie A Brazil,tôn trọng,tài năng trẻ
    
    ### Ví dụ 3:
    **Văn bản:** "Chiều 21-10, tại thành phố Hồ Chí Minh, Tạp chí Gia đình Việt Nam phối hợp với Công đoàn Cục Báo chí - Bộ Thông
    tin và Truyền thông, Hội Nhà báo Việt Nam tổ chức họp báo công bố Giải bóng đá các cơ quan báo chí toàn quốc - Press Cup 2024
    khu vực phía Nam. Phát biểu tại họp báo, nhà báo Trần Trọng Dũng, Phó Chủ tịch Hội Nhà báo Việt Nam phụ trách khu vực phía Nam
    , Trưởng ban chỉ đạo Giải cho biết, Giải Bóng đá các Cơ quan Báo chí toàn quốc Press Cup lần thứ 8 năm 2024 khu vực phía Nam sẽ
    chính thức khai mạc lúc 7h30 ngày 23-10 tại Sân vận động Tao Đàn (Số 1 Huyền Trân Công Chúa, quận 1, thành phố Hồ Chí Minh). 
    Theo Ban tổ chức, ngoài các đội bóng đến từ các cơ quan báo chí đóng trên địa bàn thành phố Hồ Chí Minh như: Báo Sài Gòn Giải
    Phóng, Người Lao Động, Đài tiếng nói Nhân dân thành phố Hồ Chí Minh VOH, Đài truyền hình thành phố, Trung tâm truyền hình         Việt Nam tại thành phố Hồ Chí Minh (VTV9)…, năm nay có sự tham dự của đội bóng đến từ Đài Phát thanh và Truyền hình Tây Ninh.
    12 đội bóng sẽ chia thành 3 bảng và thi đấu các lượt trận ở vòng bảng để tìm ra 2 đội mạnh nhất tham dự vòng chung kết toàn
    quốc sẽ diễn ra tại tỉnh Thái Nguyên vào tháng 11 tới. Theo Ban tổ chức, Giải đấu có ý nghĩa đẩy mạnh phong trào rèn luyện
    thể dục thể thao trong cán bộ, nhân viên, phóng viên, biên tập viên, nhà báo tại các cơ quan báo chí từ Trung ương đến
    địa phương. Qua đó, Giải tạo sân chơi, nâng cao thể chất, đời sống văn hóa tinh thần, tăng cường sự giao lưu, học
    hỏi giữa cán bộ, nhân viên, phóng viên, biên tập viên, các nhà báo trong mỗi cơ quan báo chí và giữa các cơ quan báo
    chí với nhau. Cơ cấu giải thưởng mùa giải Press Cup 2024: 1/ Giải Nhất: Cúp, huy chương vàng, cờ, 15 triệu đồng 
    2/ Giải Nhì: Huy chương bạc, cờ, 8 triệu đồng 3/ Giải Ba: Huy chương đồng, cờ và 5 triệu đồng 4/ Cầu thủ ghi nhiều 
    bàn thắng nhất giải: Cờ, 2 triệu đồng 5/ Thủ môn xuất sắc nhất: Cờ, 2 triệu đồng 6/ Tổ trọng tài xuất sắc: Cờ,
    2 triệu đồng 7/ Giải phong cách: Cờ, 2 triệu đồng 	
    **Nhãn:** Press Cup 2024,giải bóng đá,cơ quan báo chí,TP Hồ Chí Minh,Công đoàn Cục Báo chí,Trần Trọng Dũng,sân vận động Tao Đàn,Press Cup,khai mạc,vòng loại,toàn quốc,Hội Nhà báo Việt Nam,phía Nam,Huyền Trân,VTV9,bốc thăm chia bảng,giải đấu,nhà báo,biên tập viên,từ Trung ương đến địa phương
    ### Văn bản cần phân loại:
    {text}
    """
                
                # Gọi API Groq
                response = generate_answer_groq(prompt)
                label_str = response if response else "Không xác định"
                writer.writerow([label_str])

                print(f"Bài báo {idx} hoàn thành. Nhãn: {label_str}")
                
                # Giới hạn tốc độ để tránh vượt quá rate limit
                time.sleep(pause_seconds)
        
        print(f"Gán nhãn hoàn tất. Kết quả được lưu tại {output_file}")

    except FileNotFoundError:
        print(f"Không tìm thấy file đầu vào: {input_file}")
    
    except Exception as e:
        print(f"Lỗi khi xử lý file: {e}")


def clean_file(file_path, output_path):
    # Đọc tệp đầu vào
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Định nghĩa các tiền tố cần xóa
        prefixes_to_remove = [
            "Here are the keywords (labels) for the given text:",
            "Here are the keywords (nhãn) for the given text:", 
            "Here is the generated label:",
            "Here is the output:",
            "** Nhãn:** ",
            "Here are the keywords (nhãn) that reflect the main content of the text:", # Xóa tiền tố này
            "Here are the keywords for the given text:",          # Xóa thêm tiền tố này
            "**Nhãn:**",
            "**Nhãn:** "                                            # Xóa tiền tố này
        ]

        cleaned_lines = []
        for line in lines:
            cleaned_line = line
            # Loại bỏ các tiền tố
            for prefix in prefixes_to_remove:
                cleaned_line = cleaned_line.replace(prefix, "")
        
            # Loại bỏ dấu nháy kép (")
            cleaned_line = cleaned_line.replace('"', '')  # Loại bỏ dấu nháy kép

            # Loại bỏ khoảng trắng thừa
            cleaned_line = cleaned_line.strip()

            # Kiểm tra nếu dòng không trống và không chỉ chứa một dấu "
            if cleaned_line and cleaned_line != '"':
                cleaned_lines.append(cleaned_line)

            # Lưu nội dung đã làm sạch vào tệp đầu ra
            with open(output_path, 'w', encoding='utf-8') as output_file:
                output_file.write('\n'.join(cleaned_lines))


# if __name__ == "__main__":
#     try:
#         for i in range(2, 880):
#             input_path = f"C:/Users/admin/Desktop/articles_training/LLAMA/ID/ID_{i}.tsv"
#             output_path = f"C:/Users/admin/Desktop/articles_training/LLAMA/Result/result_{i}.tsv"
#             print(f"Đang xử lý file số {i}: {input_path}")
#             generate_labels(input_path, output_path)
#             clean_file(output_path, output_path)
#             print(f"Hoàn thành file số {i}. Kết quả: {output_path}")
#     except KeyboardInterrupt:
#         print("Người dùng đã dừng chương trình bằng Ctrl+C. Những file đã xử lý trước đó đã được lưu.")
# if not os.path.exists(input_path):
#         print(f"File does not exist: {input_path}")
if __name__ == "__main__":
    try:
        for i in range(6, 880):
            input_path = f"C:/Users/admin/Desktop/articles_training/LLAMA/ID/ID_{i}.tsv"
            output_path = f"C:/Users/admin/Desktop/articles_training/LLAMA/Result/result_{i}.tsv"
            print(f"Đang xử lý file số {i}: {input_path}")
            
            # Kiểm tra sự tồn tại của file input
            if not os.path.exists(input_path):
                print(f"File does not exist: {input_path}")
                continue

            generate_labels(input_path, output_path)
            clean_file(output_path, output_path)
            print(f"Hoàn thành file số {i}. Kết quả: {output_path}")

            # Nếu i chia hết cho 5, dừng 60 giây
            print("Đã xử lý 1 file, tạm dừng 60 giây để tránh vượt giới hạn tốc độ...")
            time.sleep(180)

    except KeyboardInterrupt:
        print("Người dùng đã dừng chương trình bằng Ctrl+C. Những file đã xử lý trước đó đã được lưu.")



Đang xử lý file số 6: C:/Users/admin/Desktop/articles_training/LLAMA/ID/ID_6.tsv
Bài báo 1 hoàn thành. Nhãn: **Nhãn:** bầu cử Mỹ,Donald Trump,Kamala Harris,cuộc bầu cử tổng thống,cuộc bầu cử 2020,cuộc bầu cử 2024,kiểm phiếu,kết quả cuối cùng,NBC News,CNN,C-Span
Bài báo 2 hoàn thành. Nhãn: bóng chuyền,Trần Thị Thanh Thúy,CEV Cup,Thổ Nhĩ Kỳ,Kuzeyboru,cúp châu Âu,huấn luyện viên,ghi điểm quyết định,trận thắng,vận động viên Việt Nam
Bài báo 3 hoàn thành. Nhãn: Dự án cầu,cây cầu,xây dựng,phương án,tim cầu,tháng 12 năm 2025.
Bài báo 4 hoàn thành. Nhãn: **Nhãn:** Chính phủ Nhật Bản,ngân sách bổ sung,Shigeru Ishiba,cử tri,đảng đối lập,phúc lợi,tăng giá.
Bài báo 5 hoàn thành. Nhãn: giao thông,học sinh,văn hóa giao thông,đội mũ bảo hiểm,chở quá số người,tín hiệu đèn giao thông,xử lý,tiêu chuẩn
Bài báo 6 hoàn thành. Nhãn: quản lý,phát triển,dự án nhà ở xã hội,hạn chế,chính sách,đời sống
Bài báo 7 hoàn thành. Nhãn: Lương Cường,Lima,Peru,chào đón,diễn ra,sân bay quốc tế Jorge Chávez,chuẩn bị,thủ đô

In [2]:
def clean_file(file_path, output_path):
    # Đọc tệp đầu vào
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Định nghĩa các tiền tố cần xóa
    prefixes_to_remove = [
        "Here are the keywords (labels) for the given text:",
        "Here are the keywords (nhãn) for the given text:", 
        "Here is the generated label:",
        "Here is the output:",
        "** Nhãn:** ",
        "Here are the keywords (nhãn) that reflect the main content of the text:", # Xóa tiền tố này
        "Here are the keywords for the given text:",          # Xóa thêm tiền tố này
        "**Nhãn:**",
        "**Nhãn:** "                                            # Xóa tiền tố này
    ]

    cleaned_lines = []
    for line in lines:
        cleaned_line = line
        # Loại bỏ các tiền tố
        for prefix in prefixes_to_remove:
            cleaned_line = cleaned_line.replace(prefix, "")
        
        # Loại bỏ dấu nháy kép (")
        cleaned_line = cleaned_line.replace('"', '')  # Loại bỏ dấu nháy kép

        # Loại bỏ khoảng trắng thừa
        cleaned_line = cleaned_line.strip()

        # Kiểm tra nếu dòng không trống và không chỉ chứa một dấu "
        if cleaned_line and cleaned_line != '"':
            cleaned_lines.append(cleaned_line)

    # Lưu nội dung đã làm sạch vào tệp đầu ra
    with open(output_path, 'w', encoding='utf-8') as output_file:
        output_file.write('\n'.join(cleaned_lines))

# Ví dụ sử dụng:
input_file = "C:/Users/admin/Desktop/articles_training/LLAMA/Result/result_1.tsv"       # Thay bằng đường dẫn tệp của bạn
output_file = "C:/Users/admin/Desktop/articles_training/LLAMA/Result/result_1.tsv" # Thay bằng đường dẫn tệp đầu ra mong muốn

# Gọi hàm làm sạch
clean_file(input_file, output_file)

print(f"Tệp đã được làm sạch và lưu tại {output_file}")


Tệp đã được làm sạch và lưu tại C:/Users/admin/Desktop/articles_training/LLAMA/Result/result_1.tsv
